<a href="https://colab.research.google.com/github/Mattemathics/Machine-Learning-Python/blob/BR-01/TLVQ_OLIVETTI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Tangent LVQ that seems to work fine on Olivetti dataset
import math
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import scipy.linalg as splin
import scipy
import sklearn
from sklearn.datasets import fetch_olivetti_faces
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

In [ ]:
## eps = learning step, data-dim = extrinsic data dim, pn = number of prototoypes, pd = dimensions of prototype
eps= 0.01
data_dim=64*64
c=40
pn = 40
pd = 3

In [ ]:
####define the dataset
faces =fetch_olivetti_faces(data_home=None, shuffle=True, random_state=0, download_if_missing=True, return_X_y=False)
images = faces.images
features = faces.data
targets = faces.target
train_features, test_features, train_targets, test_targets = train_test_split(
        features, targets,
        train_size=0.8,
        test_size=0.2,
        # random but same for all run, also accuracy depends on the
        # selection of data e.g. if we put 10 then accuracy will be 1.0
        # in this example
        random_state=23,
        # keep same proportion of 'target' in test and target data
        stratify=targets
    )

idx = np.argsort(train_targets)
#idx = np.squeeze(idx)
train_features = train_features[idx]
train_targets = train_targets[idx]

train_features = tf.keras.utils.normalize(train_features, axis=1)
test_features = tf.keras.utils.normalize(test_features, axis=1)

downloading Olivetti faces from https://ndownloader.figshare.com/files/5976027 to /root/scikit_learn_data


In [ ]:
####Functions 

def orthnomalization(W):
    for i in range(pn):
        W[i] = np.transpose(splin.orth(np.transpose(W[i])))

def tanDist(x,W,v):
    dist = np.random.uniform(low=0, high=0.5, size=[pn])
    for i in range(pn):
        dist[i] = np.linalg.norm(x-v[i])**2 - np.linalg.norm(np.matmul(W[i], x-v[i]))**2
    return dist

In [ ]:
#define protototypes
W = np.random.uniform(low=0, high=0.5, size=[pn, pd, data_dim])
orthnomalization(W)
v = np.random.uniform(low=0, high=0.01, size=[pn, data_dim])


#initializing the bias prototypes 
for i in range(320):
    v[train_targets[i]] = train_features[i] + v[train_targets[i]]

for i in range(c):
    v[i] = v[i]/np.linalg.norm(v[i])

pca = PCA()
for i in range(c):
  one = train_features[train_targets == i] - v[i]
  pca.fit(one)
  two = pca.components_
  three = two[0:pd,:]
  W[i] =three

In [ ]:
##Training part tangent LVQ
for tot in range(10):
  for ii in range(500):
      lab = np.random.randint(0, 320, dtype=int)
      x = train_features[lab]
      y = train_targets[lab]
      mat = np.identity(data_dim) - np.matmul(np.transpose(W[y]), W[y])
      diff =  x - v[y]
      diff = np.reshape(diff,(data_dim,1))
      W[y] = W[y] + 2* eps *np.transpose(np.matmul(np.matmul(diff,np.transpose(diff)),np.transpose(W[y])))
      v[y] = v[y] + 2* eps * np.reshape(np.matmul(mat,diff),(data_dim,))
      W[y] = np.transpose(splin.orth(np.transpose(W[y])))
  ###evaluation
  acc = 0
  for i in range(80):
      x = test_features[i]
      y = test_targets[i]
      dist = tanDist(x,W,v)
      la = np.argmin(dist)
      if la == y:
          acc = acc + 1

  print("accuracy is")
  print(acc/80)

accuracy is
0.9625
accuracy is
0.9625
accuracy is
0.9625
accuracy is
0.9625
accuracy is
0.9625
accuracy is
0.9625
accuracy is
0.9625
accuracy is
0.9625
accuracy is
0.9625
accuracy is
0.9625


In [ ]:
###evaluation
acc = 0
for i in range(80):
    x = test_features[i]
    y = test_targets[i]
    dist = tanDist(x,W,v)
    la = np.argmin(dist)
    if la == y:
        acc = acc + 1

print("accuracy is")
print(acc/80)

accuracy is
0.9625


In [ ]:
fig = plt.figure(figsize=(10,7))
alph = ['a','b','c','d','e','f','g','h','i','j']
alph = alph[0:c]
x = np.array(range(c))
x1 = np.array(range(2))

for i in range(3):
    img = np.reshape(W[2,i], (64, 64))
    fig.add_subplot(2, 10 + 1, i + 1)
    plt.imshow(img, cmap=plt.cm.binary)
    plt.axis('off')
    plt.title(alph[i])

for i in range(10):
    im = np.reshape(v[i], (64, 64))
    fig.add_subplot(2, 10 + 1, i + 12)
    plt.imshow(im, cmap=plt.cm.binary)
    plt.yticks(x, alph)
    plt.xticks(x1, ['m','-m'])
    plt.title(i)

plt.tight_layout()
plt.show()

In [ ]:
promat = np.random.uniform(low=0, high=0.5, size=[40, 40])
free = np.random.uniform(low=0, high=0.5, size=[40, 40])
for i in range(40):
  for j in range(40):
    matr = np.concatenate((W[i],W[j]),axis=0)
    free[i,j] = 2*np.linalg.matrix_rank(matr) - np.linalg.matrix_rank(W[i]) - np.linalg.matrix_rank(W[j])
    promat[i,j] = (np.linalg.norm(v[i]-v[j])**2 - np.linalg.norm(np.matmul(matr, v[i]-v[j]))**2)**2
    #print(a)
    #print(np.linalg.norm(np.matmul(matr, (v[i]-v[j+i+1]))))
print(promat)
print(free)

[[0.         0.00080117 0.00057341 ... 0.00045594 0.00062532 0.00036225]
 [0.00080117 0.         0.00272112 ... 0.00321004 0.00254601 0.0027276 ]
 [0.00057341 0.00272112 0.         ... 0.000454   0.00064373 0.00016442]
 ...
 [0.00045594 0.00321004 0.000454   ... 0.         0.00112018 0.00010992]
 [0.00062532 0.00254601 0.00064373 ... 0.00112018 0.         0.00052515]
 [0.00036225 0.0027276  0.00016442 ... 0.00010992 0.00052515 0.        ]]
[[0. 6. 6. ... 6. 6. 6.]
 [6. 0. 6. ... 6. 6. 6.]
 [6. 6. 0. ... 6. 6. 6.]
 ...
 [6. 6. 6. ... 0. 6. 6.]
 [6. 6. 6. ... 6. 0. 6.]
 [6. 6. 6. ... 6. 6. 0.]]
